In [2]:
import torch
from torch import nn
import os
import cv2
import numpy as np

from torch.utils.data import DataLoader
from torch import optim

from tqdm.notebook import tqdm

In [1]:
from torchvision import io

In [3]:
DATA_FOLDER = 'tiles_ready/tiles_1m_23d_1h/'

IMGS_FOLDER = f'{DATA_FOLDER}/imgs/'
MASKS_FOLDER = f'{DATA_FOLDER}/masks/'

In [4]:
img_paths = np.array([IMGS_FOLDER + file for file in sorted(os.listdir(IMGS_FOLDER)) if 'png' in file])
masks_paths =  np.array([MASKS_FOLDER + file for file in sorted(os.listdir(MASKS_FOLDER)) if 'png' in file])

In [5]:
SEED = 0

In [6]:
torch.manual_seed(SEED)
np.random.seed(SEED)

In [7]:
import torch

class Dataset(torch.utils.data.Dataset):
    def __init__(self, imgs, labels, device):
        self.labels = labels
        self.imgs = imgs
        self.device = device

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, index):
        # Select sample
        img_path = self.imgs[index]
        label_path = self.labels[index]
            
        # Load data and get label
        X = io.read_image(img_path).type(torch.float).to(self.device)
        y = torch.tensor(cv2.imread(label_path, cv2.IMREAD_GRAYSCALE), dtype=torch.float, device=self.device)

        return X, y

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
train_ind, val_ind = train_test_split(np.arange(len(img_paths)), test_size=0.2)

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [12]:
# device = 'cpu'

In [13]:
train_dataset = Dataset(img_paths[train_ind], masks_paths[train_ind], device)
val_dataset = Dataset(img_paths[val_ind], masks_paths[val_ind], device)

In [14]:
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_loader =  DataLoader(val_dataset, batch_size=4, shuffle=True)

## Обучение модельки

In [6]:
import segmentation_models_pytorch as smp

In [15]:
segm_net = nn.DataParallel(smp.Unet('timm-efficientnet-b8', encoder_weights='imagenet', activation='sigmoid')).to(device)

In [16]:
torch.manual_seed(SEED)
np.random.seed(SEED)

In [17]:
optimizer = optim.Adam(segm_net.parameters())

In [18]:
NUM_EPOCHS = 2

In [19]:
loss_f = nn.BCEWithLogitsLoss()

In [20]:
def val_model(val_loader, model, thresh=0.5):
    tp = 0
    all_p = 0

    with torch.no_grad():
        for i, eval_batch in enumerate(tqdm(val_loader)):
            input_t, mask = eval_batch[0], eval_batch[1]
            pred = segm_net(input_t)

            pred = (pred > 0.5).type(torch.float)

            res = pred.squeeze(1) == mask
            res = res.type(torch.float)

            tp += res.sum()
            all_p += res.shape[0]*res.shape[1]*res.shape[2]
    
    return (tp/all_p).item()

In [23]:
for i in range(NUM_EPOCHS):
    for i, train_batch in enumerate(tqdm(train_loader)):
        optimizer.zero_grad()

        input_tensor, mask = train_batch[0], train_batch[1]
        pred = segm_net(input_tensor)

        loss = loss_f(pred.squeeze(1), mask)

        loss.backward()
        optimizer.step()

        torch.cuda.empty_cache()
        
        if i % 300 == 0:
            print(loss.item())
    
    print(val_model(val_loader, segm_net))

0.9203220009803772
0.6941013932228088
0.6933915615081787
0.6932647228240967
0.6932274103164673
0.6932029128074646




tensor(0.9650, device='cuda:0')


0.6932005882263184
0.6931790113449097
0.6931731700897217
0.6931685209274292
0.6931637525558472
0.6931593418121338




tensor(0.9650, device='cuda:0')
